In [3]:
import numpy as np 
from scipy import sparse 
from scipy.sparse.linalg import lsqr, cg, eigsh
#from meshplot import plot, subplot, interact
import matplotlib.pyplot as plt 
import argparse
#import igl
import os
import scipy as sp
import scipy.sparse.linalg as sla
#from trimesh import load_off, save_off

In [4]:
!conda install -c conda-forge igl

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
v0, f0 = igl.read_triangle_mesh(os.path.join("non-rigid-world", "cat0.obj"))
v1, f1 = igl.read_triangle_mesh(os.path.join("non-rigid-world", "cat1.obj"))

In [ ]:
K_g0 = igl.gaussian_curvature(v0,f0)
K_g1 = igl.gaussian_curvature(v1,f1)

In [ ]:
C = np.random.rand(10,10)

In [ ]:
plot(v0, f0, K_g0)

In [ ]:
plot(v1, f1, K_g1)

In [ ]:
def get_connected_points(v,f):
    connected_points = []
    for k,i in enumerate(range(v.shape[0])):
        points = []
        for j in f:
            if i in j:
                points += list(j)
        connected_points.append(list(set(points)))
        
    return connected_points

In [ ]:
connected_points_0 = get_connected_points(v0,f0)
connected_points_1 = get_connected_points(v1,f1)

In [ ]:
def get_cotan_laplacian(VPos, ITris, anchorsIdx = [], anchorWeights = 1):
    """
    Quickly compute sparse Laplacian matrix with cotangent weights and Voronoi areas
    by doing many operations in parallel using NumPy
    
    Parameters
    ----------
    VPos : ndarray (N, 3) 
        Array of vertex positions
    ITris : ndarray (M, 3)
        Array of triangle indices
    anchorsIdx : list
        A list of vertex indices corresponding to the anchor vertices 
        (for use in Laplacian mesh editing; by default none)
    anchorWeights : float
    

    Returns
    -------
    L : scipy.sparse (NVertices+anchors, NVertices+anchors)
        A sparse Laplacian matrix with cotangent weights
    """
    N = VPos.shape[0]
    M = ITris.shape[0]
    #Allocate space for the sparse array storage, with 2 entries for every
    #edge for eves ry triangle (6 entries per triangle); one entry for directed 
    #edge ij and ji.  Note that this means that edges with two incident triangles
    #will have two entries per directed edge, but sparse array will sum them 
    I = np.zeros(M*6)
    J = np.zeros(M*6)
    V = np.zeros(M*6)
    
    #Keep track of areas of incident triangles and the number of incident triangles
    IA = np.zeros(M*3)
    VA = np.zeros(M*3) #Incident areas
    VC = 1.0*np.ones(M*3) #Number of incident triangles
    
    #Step 1: Compute cotangent weights
    for shift in range(3): 
        #For all 3 shifts of the roles of triangle vertices
        #to compute different cotangent weights
        [i, j, k] = [shift, (shift+1)%3, (shift+2)%3]
        dV1 = VPos[ITris[:, i], :] - VPos[ITris[:, k], :]
        dV2 = VPos[ITris[:, j], :] - VPos[ITris[:, k], :]
        Normal = np.cross(dV1, dV2)
        #Cotangent is dot product / mag cross product
        NMag = np.sqrt(np.sum(Normal**2, 1))
        cotAlpha = np.sum(dV1*dV2, 1)/NMag
        I[shift*M*2:shift*M*2+M] = ITris[:, i]
        J[shift*M*2:shift*M*2+M] = ITris[:, j] 
        V[shift*M*2:shift*M*2+M] = cotAlpha
        I[shift*M*2+M:shift*M*2+2*M] = ITris[:, j]
        J[shift*M*2+M:shift*M*2+2*M] = ITris[:, i] 
        V[shift*M*2+M:shift*M*2+2*M] = cotAlpha
        if shift == 0:
            #Compute contribution of this triangle to each of the vertices
            for k in range(3):
                IA[k*M:(k+1)*M] = ITris[:, k]
                VA[k*M:(k+1)*M] = 0.5*NMag
    
    #Step 2: Create laplacian matrix
    L = sparse.coo_matrix((V, (I, J)), shape=(N, N)).tocsr()
    #Create the diagonal by summing the rows and subtracting off the nondiagonal entries
    L = sparse.dia_matrix((L.sum(1).flatten(), 0), L.shape) - L
    #Scale each row by the incident areas TODO: Fix this
    """
    Areas = sparse.coo_matrix((VA, (IA, IA)), shape=(N, N)).tocsr()
    Areas = Areas.todia().data.flatten()
    Areas[Areas == 0] = 1
    Counts = sparse.coo_matrix((VC, (IA, IA)), shape=(N, N)).tocsr()
    Counts = Counts.todia().data.flatten()
    RowScale = sparse.dia_matrix((3*Counts/Areas, 0), L.shape)
    L = L.T.dot(RowScale).T
    """
    
    #Step 3: Add anchors
    L = L.tocoo()
    I = L.row.tolist()
    J = L.col.tolist()
    V = L.data.tolist()
    I = I + list(range(N, N+len(anchorsIdx)))
    J = J + anchorsIdx
    V = V + [anchorWeights]*len(anchorsIdx)
    L = sparse.coo_matrix((V, (I, J)), shape=(N+len(anchorsIdx), N)).tocsr()
    return L

def get_hks(VPos, ITris, K, ts):
    """
    Given a triangle mesh, approximate its curvature at some measurement scale
    by recording the amount of heat that remains at each vertex after a unit impulse
    of heat is applied.  This is called the "Heat Kernel Signature" (HKS)

    Parameters
    ----------
    VPos : ndarray (N, 3)
        Array of points in 3D
    ITris : ndarray (M, 3)
        Array of triangles connecting points, pointing to vertex indices
    K : int
        Number of eigenvalues/eigenvectors to use
    ts : ndarray (T)
        The time scales at which to compute the HKS
    
    Returns
    -------
    hks : ndarray (N, T)
        A array of the heat kernel signatures at each of N points
        at T time intervals
    """
    L = get_cotan_laplacian(VPos, ITris)
    (eigvalues, eigvectors) = eigsh(L, K,which='LM', sigma = 0)
    res = (eigvectors[:, :, None]**2)*np.exp(-eigvalues[None, :, None]*ts.flatten()[None, None, :])
    return np.sum(res, 1)

hks = get_hks(v0,f0,10,np.arange(98))

In [ ]:
def init_C(n_vec):
    return np.random.rand(n_vec,n_vec)

def Signature_Hist(v,f,neighborhood):
    sig_hist_desc = []
    for i,neigh in enumerate(neighborhood):
        points = v[neigh]
        normals = [np.cross(points[j],points[j+1]) for j,_ in enumerate(points) if j<len(points)-1]
        sig_hist = [np.dot(v[i],normals[j])/ (np.linalg.norm(v[i])*np.linalg.norm(normals[j])) for j,_ in enumerate(normals)]
        sig_hist_desc.append(np.linalg.norm(sig_hist))
        
    return sig_hist_desc


def functional_map_coeff(vertices,faces,n_vec):
    guass_curv = Gaussian_Curvature_coeff(vertices,faces)
    (evals,evecs) = get_eval_evec(vertices,faces,n_vec)
    return guass_curv.dot(evecs)

def functional_map_coeff_hist(vertices,faces,connected_points,n_vec):
    guass_curv = np.array(Signature_Hist(vertices,faces,connected_points))
    (evals,evecs) = get_eval_evec(vertices,faces,n_vec)
    return guass_curv.dot(evecs)

def Gaussian_Curvature_coeff(vertices,faces):
    return igl.gaussian_curvature(vertices,faces)
    
def get_eval_evec(vertices,faces,n_vec):
    L = get_cotan_laplacian(vertices,faces)
    (evals,evecs) = eigsh(L,n_vec,which='LM', sigma = 0)
    return (evals,evecs)

#def cotan_laplacian(vertices,faces):
    #return igl.cotmatrix(vertices,faces)

def get_loss(C,vertices_M, faces_M,vertices_N,faces_N,n_vecs=10):
    A = functional_map_coeff(vertices_M, faces_M,n_vecs)
    B = functional_map_coeff(vertices_N,faces_N,n_vecs)
    A_eval,_ = get_eval_evec(vertices_M,faces_M,n_vecs)
    B_eval,_ = get_eval_evec(vertices_N,faces_N,n_vecs)

    return np.linalg.norm(C@A-B) #+ np.linalg.norm(C@A_eval- B_eval@C)) * 0.5

def grad_c(C,A,B,A_eval, B_eval):
    return ((C@A-B) @ A) + C * np.linalg.norm(np.square(A_eval- B_eval))

In [ ]:
#C = init_C(10) #step 1 : Initialize Functional Map Matrix
import scipy as sp
#Step 2 : Load the input and target mesh
v0, f0 = igl.read_triangle_mesh(os.path.join("non-rigid-world", "cat0.obj"))
v1, f1 = igl.read_triangle_mesh(os.path.join("non-rigid-world", "cat1.obj"))

#Step 3 : Initialize first loss
#loss = get_loss(C,v0,f0,v1,f1,10)

#Step 5 : Get the Eigen vectors and Eigen values stored in some variable
A_eval,A_evec = get_eval_evec(v0, f0,10)
B_eval,B_evec = get_eval_evec(v1,f1,10)

#Step 4 : Get Coefficients for Eigen Vectors for input and target
guass_curve_fun_0 = functional_map_coeff(v0, f0,10)
guass_curve_fun_1 = functional_map_coeff(v1,f1,10)

sig_hist_fun_0 = functional_map_coeff_hist(v0,f0,connected_points_0,10)
sig_hist_fun_1 = functional_map_coeff_hist(v1,f1,connected_points_1,10)

hks_0 = A_evec.T.dot(get_hks(v0, f0, 10, np.arange(98)))
hks_1 = B_evec.T.dot(get_hks(v1, f1, 10, np.arange(98)))

print((guass_curve_fun_0.shape,sig_hist_fun_0.shape,hks_0.shape))
A = np.column_stack((guass_curve_fun_0.reshape(10,1),sig_hist_fun_0.reshape(10,1),hks_0)).T
B = np.column_stack((guass_curve_fun_1.reshape(10,1),sig_hist_fun_1.reshape(10,1),hks_1)).T



#Step 6 : Get the functional matching from nearest neighbor or C l2 optimization.
C, res, rnk, s = sp.linalg.lstsq(A,B)

In [ ]:
def p2p_from_fmap(a,C,A_evec,B_evec):
    f = A_evec.dot(a.T)
    e = C @ a.T
    g = B_evec.dot(e)
    
    for k,vec in enumerate(f):
        max_val = vec.max()
        vec = np.where(vec==max_val,1,0)
        f[k] = vec
    
    for k,vec in enumerate(g):
        max_val = vec.max()
        vec = np.where(vec==max_val,1,0)
        g[k] = vec
    
    sns.heatmap(f.dot(g.T))
    plt.title('Correlation matrix among f and g')
    plt.show()
    
    return f,g

In [ ]:
f,g = p2p_from_fmap(A,C,A_evec,B_evec)

In [ ]:
sns.set_theme()

plt.figure(figsize=(24,8))
plt.plot(np.arange(100),f[0,:])
plt.show()